# Super image resolution

Enlarge images using a neural network

In [ ]:
#@title ▶ Install required libraries and download the pretrained models

# Clone SwinIR
!git clone https://github.com/JingyunLiang/SwinIR.git
%cd SwinIR
!pip install -q timm

# Download the pre-trained models
!mkdir pretrained_models
!wget -q --show-progress https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN.pth -P pretrained_models
!wget -q --show-progress https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth -P pretrained_models
!wget -q --show-progress https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth -P pretrained_models

In [ ]:
#@title ▶ Connect google drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ▶ Enlarge images

#@markdown This should be the path of the folder inside Drive. The best way is to find it in the file explorer on the left, click on the 3 points, use the option "copy path" and paste the value here.

#@markdown >⚠ The folder must contain only images, if not it may not work properly.
#@markdown > Enlarged images will be exported in a subfolder of this path. Move or remove this folder if you want to process the images again.
input_path = "/content/drive/MyDrive/BAU-DD/enlarge" #@param {type:"string"}

#@markdown Scaling factor (enlarge the images x2 or x4)

#@markdown >⚠ 2 only works with large: False
scale = 4 #@param [2, 4] {type:"raw"}

#@markdown Large to true should give better results, but will be slower.
large = True #@param ["False", "True"] {type:"raw"}

#@markdown Change it to True if you are processing "big" images (bigger than 700px or 800px), especially at 4 scaling factor.
patch_wise = False #@param ["False", "True"] {type:"raw"}

#@markdown 📌 Best quality is with `large: True` and `patch_wise: False` but it may fail due to memory requirements. It it fails try with `large: True` and `patch_wise: True`, `large: False` and `patch_wise: False` or finally `large: False` and `patch_wise: True` (in this order).

import os
import glob
import shutil

large_args = ''
patch_args = ''
if large:
  large_args = '--large_model'
  intermediate_path = 'results/swinir_real_sr_x4_large'
  model = '003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
  if patch_wise:
    patch_args = '--tile 640'
else:
  intermediate_path = 'results/swinir_real_sr_x%s' % scale
  if scale == 2:
    model = '003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x2_GAN.pth'
  else:
    model = '003_realSR_BSRGAN_DFO_s64w8_SwinIR-M_x4_GAN.pth'
  if patch_wise:
    patch_args = '--tile 800'

! time python main_test_swinir.py --task real_sr --model_path pretrained_models/$model --folder_lq $input_path --scale $scale $large_args $patch_args 

output_path = os.path.join(input_path, "resize-x%s" % scale)
if not os.path.isdir(output_path):
    os.makedirs(output_path)

for intermediate_file in sorted(glob.glob(os.path.join(intermediate_path, '*.png'))):
  output_file = os.path.join(output_path, os.path.basename(intermediate_file))
  if os.path.isfile(output_file):
    os.remove(output_file)
  shutil.move(intermediate_file, output_path)

#from google.colab import output
#output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

# Credits

Taller Estampa https://tallerestampa.com / https://github.com/estampa

### Based on

[SwinIR](https://github.com/JingyunLiang/SwinIR) notebook https://colab.research.google.com/gist/JingyunLiang/a5e3e54bc9ef8d7bf594f6fee8208533/swinir-demo-on-real-world-image-sr.ipynb
